# Ejemplo de ETL: Limpieza de Nulos y Outliers

Aquí realizaremos un flujo ETL típico:
1. Carga de datos del Padrón Electoral.
2. Detección y limpieza de valores nulos.
3. Detección y filtrado de outliers (ejemplo: edades fuera de rango lógico).
4. Transformación de columnas y guardado.


In [ ]:
from pyspark.sql.functions import col, when

# Cargar padrón electoral
df = spark.read.csv("/dbfs/FileStore/padron/PADRON_COMPLETO.csv", header=True, inferSchema=True)

# 1. Inspección inicial
df.printSchema()
df.select([col for col in df.columns]).show(5)

# 2. Detección de nulos
for c in df.columns:
    print(c, "->", df.filter(col(c).isNull()).count(), "nulos")

# 3. Limpieza: Eliminar filas con cédula o nombre nulo
df_clean = df.dropna(subset=["CEDULA", "NOMBRE"])

# 4. Supongamos que tenemos una columna EDAD y filtramos outliers
# (Si no hay, simular)
from pyspark.sql.functions import rand
df_clean = df_clean.withColumn("EDAD", (rand()*90+10).cast("int"))  # Simular edad

# Filtrar edades <18 o >100
df_clean = df_clean.filter((col("EDAD") >= 18) & (col("EDAD") <= 100))

# 5. Transformación: agregar columna "mayor_de_edad"
df_clean = df_clean.withColumn("mayor_de_edad", when(col("EDAD") >= 18, True).otherwise(False))

df_clean.select("CEDULA", "NOMBRE", "EDAD", "mayor_de_edad").show(5)

# Guardar resultado
df_clean.write.mode("overwrite").csv("/dbfs/FileStore/padron/limpio", header=True)
